[Classificação de frases por setor](https://handtalk.notion.site/Classifica-o-de-frases-por-setor-18c80adbbf874c519c9efe19678ac4c1)

In [139]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.graph_objects as go
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabriel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [140]:
df = pd.read_csv("dataset.csv")

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  521 non-null    object
 1   category  521 non-null    object
dtypes: object(2)
memory usage: 8.3+ KB


In [142]:
df

,sentence,category
0,"Auxílio-Doença Previdenciário, Benefícios em E...",orgão público
1,"PAGAR TODAS AS CONTAS EM ATRASO R$1.290,90.",finanças
2,Então encontraremos na próxima aula.,educação
3,Veja os resultados da categoria de ofertas do ...,indústrias
4,"Além disso, a embalagem é reutilizável e 100% ...","indústrias,varejo"
...,...,...
516,"Selecione o local de estudo, curso sem encontr...",educação
517,ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS E...,"educação,orgão público"
518,Empresas e órgãos públicos,orgão público
519,DGE – Departamento de Gestão Estratégica Metas...,orgão público


In [143]:
col0 = np.unique(df["category"].str.split(",").apply(pd.Series).loc[:, 0])
col1 = np.unique(df["category"].str.split(",").apply(pd.Series).loc[:, 1].dropna())
cat = np.unique(np.append(col1, col0))
num_classes = len(cat)

In [144]:
num_classes

5

The training dataset contains texts that are categorized into one or more of six distinct classes: 'educação', 'finanças', 'indústrias', 'orgão público', and 'varejo'. This setup constitutes a multi-label classification challenge.

## Preprocessing

First, we'll divide the dataset into training and testing sets. This ensures that the model is trained on a subset of the data and evaluated on a separate set it hasn't seen before, allowing for a fair assessment of its performance. This step is crucial in preventing data leakage and ensuring that our evaluation metrics accurately reflect the model's ability to generalize to new data.

In [145]:
train, test = train_test_split(df, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

### One hot encoding targe variable

Second, we'll apply one-hot encoding to the target column. This process will transform each category into a separate column, where a category's presence or absence in a sample is represented by 1 or 0, respectively.

In [146]:
train["category"] = train["category"].str.split(",")
test["category"] = test["category"].str.split(",")

mlb = MultiLabelBinarizer()

one_hot_encoded_train = mlb.fit_transform(train['category'])
one_hot_train_df = pd.DataFrame(one_hot_encoded_train, columns=mlb.classes_)
train = pd.concat([train, one_hot_train_df], axis=1).drop('category', axis=1)

one_hot_encoded_test = mlb.transform(test['category'])
one_hot_test_df = pd.DataFrame(one_hot_encoded_test, columns=mlb.classes_)
test = pd.concat([test, one_hot_test_df], axis=1).drop('category', axis=1)

In [147]:
# check missing values in numeric columns
train.describe()

,educação,finanças,indústrias,orgão público,varejo
count,416.000000,416.000000,416.000000,416.000000,416.000000
mean,0.218750,0.146635,0.221154,0.319712,0.197115
std,0.413896,0.354167,0.415524,0.466926,0.398299
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [148]:
# Verifying if there is any blank sentence
print(df[df["sentence"] == ""])
print("\n")
print(df[df["sentence"] == " "])

Empty DataFrame
Columns: [sentence, category]
Index: []


Empty DataFrame
Columns: [sentence, category]
Index: []


### Exploratory Data Analysis

Let's plot a histogram to verify the distribution of sentence length

In [149]:
train['sentence_length'] = train['sentence'].apply(lambda x: len(str(x)))

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=train["sentence_length"],
    marker_color='#7860bd',
))
fig.update_layout(
    title={
    'text': "<b>Sentence length histogram</b>",
    'font' : dict(size=26, color='black', family='Helvetica'),
    'y':0.95,
    'x':0.5},
    height=550, width=1100,
    font_family='Helvetica',
    font_color='black',
    font_size=16,
    plot_bgcolor='white',
)

fig.show()

In [150]:
df

,sentence,category
0,"Auxílio-Doença Previdenciário, Benefícios em E...",orgão público
1,"PAGAR TODAS AS CONTAS EM ATRASO R$1.290,90.",finanças
2,Então encontraremos na próxima aula.,educação
3,Veja os resultados da categoria de ofertas do ...,indústrias
4,"Além disso, a embalagem é reutilizável e 100% ...","indústrias,varejo"
...,...,...
516,"Selecione o local de estudo, curso sem encontr...",educação
517,ESTUDANTES DA REDE MUNICIPAL VOLTAM ÀS AULAS E...,"educação,orgão público"
518,Empresas e órgãos públicos,orgão público
519,DGE – Departamento de Gestão Estratégica Metas...,orgão público


In [151]:
def category_length(category):
    length_sum = (train[category] * train["sentence_length"]).sum()
    length_mean = np.round(length_sum / train[category].sum(), 2)
    return length_sum, length_mean

In [152]:
for category in cat:
    length_sum, length_mean = category_length(category)
    print(f"Summing the length of all sentences in the category '{category}' we have {length_sum} characters.")
    print(f"The mean length of the category '{category}' is {length_mean} characters.\n")

Summing the length of all sentences in the category 'educação' we have 6537 characters.
The mean length of the category 'educação' is 71.84 characters.

Summing the length of all sentences in the category 'finanças' we have 4235 characters.
The mean length of the category 'finanças' is 69.43 characters.

Summing the length of all sentences in the category 'indústrias' we have 6256 characters.
The mean length of the category 'indústrias' is 68.0 characters.

Summing the length of all sentences in the category 'orgão público' we have 8748 characters.
The mean length of the category 'orgão público' is 65.77 characters.

Summing the length of all sentences in the category 'varejo' we have 4792 characters.
The mean length of the category 'varejo' is 58.44 characters.



In [153]:
train["sentence"] = train["sentence"].str.strip().str.lower()
test["sentence"] = test["sentence"].str.strip().str.lower()

In [154]:
train

,sentence,educação,finanças,indústrias,orgão público,varejo,sentence_length
0,pague a primeira parcela em 2022 ou parcelas d...,0,1,1,0,0,72
1,estojo em madeira coleção magnus: 30 cores + k...,0,0,0,0,1,58
2,design bruto e bonito ao mesmo tempo: este é o...,0,0,1,0,0,59
3,piloto automático em seu carro tem o controle ...,0,0,1,0,0,71
4,conheça a opção mais adequada para o seu bolso...,0,1,0,0,1,75
...,...,...,...,...,...,...,...
411,"lei 13.467/2017, reforma trabalhista",0,0,0,1,0,36
412,a prova presencial será realizada na unidade d...,1,0,0,0,0,71
413,acompanhe as ações da sptrans que contribuem p...,0,0,0,1,0,74
414,cláusula anticorrupção - instrução normativa c...,0,0,0,1,0,72


In [155]:
train = train.drop('sentence_length',axis=1)
X_train = train.sentence
X_test = test.sentence

print(X_train.shape, X_test.shape)

(416,) (105,)


### TfidfVectorizer

TfidfVectorizer stands for Term Frequency-Inverse Document Frequency. It enhances the simple count-based approach by considering not only how often a word appears in a single document but also how unique the word is across all documents in the corpus. It combines two metrics:

- Term Frequency (TF): Similar to CountVectorizer, it measures how frequently a term occurs in a document. This is normalized by dividing by the total number of words in the document to avoid bias towards longer documents.
- Inverse Document Frequency (IDF): This measures how unique or common a word is in the entire document corpus. The more documents a word appears in, the lower its IDF (and thus, its importance).

The TF-IDF score of a word increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word across the corpus. This helps to diminish the effect of frequently occurring words that don’t hold much meaningful information about the document

In [158]:
stop_words_pt = stopwords.words('portuguese')
vectorizer = TfidfVectorizer(stop_words=stop_words_pt)

X_train_matrix = vectorizer.fit_transform(X_train)
X_test_matrix = vectorizer.transform(X_test)

In [159]:
X_train_matrix

<416x1667 sparse matrix of type '<class 'numpy.float64'>'
	with 2766 stored elements in Compressed Sparse Row format>

In [160]:
X_test_matrix

<105x1667 sparse matrix of type '<class 'numpy.float64'>'
	with 426 stored elements in Compressed Sparse Row format>

## Training

In [87]:
train, test = train_test_split(df, test_size=0.2)
X_train = train.loc[:, "sentence"]
y_train = train["sentence"]
X_test = test.loc[:, "sentence"]
y_test = test["sentence"]

In [88]:
tokenizer_obj = Tokenizer()
# tokenizer_obj.fit_on_texts(df.loc[:, "sentence"])
tokenizer_obj.fit_on_texts(X_train)
max_length = max(len(s.split()) for s in X_train)
vocab_size = len(tokenizer_obj.word_index) + 1

In [89]:
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding="post")
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding="post")

In [91]:
embedding_dim = 100
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    LSTM(units=128),
    Dense(num_classes, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_split=0.2)

2024-03-19 21:30:06.304982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-19 21:30:06.325700: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10


ValueError: in user code:

    File "/home/gabriel/.pyenv/versions/3.8.0/envs/airflow_dags/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/gabriel/.pyenv/versions/3.8.0/envs/airflow_dags/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/gabriel/.pyenv/versions/3.8.0/envs/airflow_dags/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/gabriel/.pyenv/versions/3.8.0/envs/airflow_dags/lib/python3.8/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/gabriel/.pyenv/versions/3.8.0/envs/airflow_dags/lib/python3.8/site-packages/keras/src/engine/training.py", line 1139, in compute_